<a href="https://colab.research.google.com/github/anespart1/educationuse/blob/main/%EC%9D%B8%EA%B5%AC%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%8B%9C%EA%B0%81%ED%99%94_%EC%A7%80%EC%97%AD%EC%9D%B8%EA%B5%AC%EA%B5%AC%EC%A1%B0%EB%B9%84%EA%B5%90_%EC%B5%9C%EC%A2%85_20210824.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#인구구조가 비슷한/전혀다른 지역은 어디일까?
(송석리 선생님 방법 참고)

#판다스 & 플로틀리 가져오기 및 설정(숨김)

In [1]:
import pandas as pd
import numpy as np
import plotly.offline
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import BytesIO

#플로티 옵션 설정 및 준비
plotly.offline.init_notebook_mode(connected=True)
pio.renderers.default = 'colab'

#인구데이터시각화 연도 설정(변경시 Ctrl+ F10으로 여기부터 재실행)

In [2]:
#@title 연도를 입력해주세요 { run: "auto" }
target_year = 2012 #@param {type:"slider", min:2008, max:2020, step:1}
print('{}년을 선택했습니다.'.format(target_year))
url = "https://raw.githubusercontent.com/anespart1/educationuse/master/{}12_{}12_연령별인구현황_연간.xlsx".format(target_year, target_year)
data = requests.get(url).content
pop = pd.read_excel(BytesIO(data)) 

2012년을 선택했습니다.


#코드 실행부분(숨김)

In [3]:
def insert_columns(data, exist_data):
  for i in range(2):
    data.insert(0 + i, exist_data.columns[i], exist_data[exist_data.columns[i]])

In [4]:
#항목명 변환, 필요없는 행 제거, 인덱스 리셋, 행정구분 추가, 열 순서 변경
def data_adjust(data=None):
  data.columns = data.iloc[2].values.tolist()
  data = data[data['행정기관코드'] != '3611000000'] #2012년 이후 데이터의 세종시중복문제 회피
  data = data.drop(data.index[[0, 1, 2]], axis=0)
  data.reset_index(drop=True, inplace=True)
  data['행정구분'] = data['행정기관'].apply(lambda x: str(len(str(x).split())))
  data['행정구분'] = data['행정구분'].apply(lambda x:x.replace('1',"시도")).apply(lambda x:x.replace('2',"시군구")).apply(lambda x:x.replace('3',"읍면동"))
  col_list = list(data.columns)[:-1]
  col_list.insert(2, data.columns[-1])
  data = data[col_list]

  return data

In [5]:
def data_normalization(data=None, columns=None, idx=None, pre_reset='y'):
  ndata = data.transpose()
  ndata.columns = columns
  if pre_reset == 'y':
    ndata.reset_index(inplace=True)
  ndata = ndata.drop(ndata.index[idx], axis=0)
  ndata.reset_index(drop=True, inplace=True)
  for i in range(1, len(ndata.columns)):
    ndata[ndata.columns[i]] = ndata[ndata.columns[i]].apply(lambda x: int(x.replace(',', '')))
    ndata[ndata.columns[i]] = ndata[ndata.columns[i]] / ndata[ndata.columns[i]][0]
  
  return ndata

In [6]:
def prepare_data_comparing(data=None):
  ndata = data.transpose().copy()
  ndata.columns = ndata.iloc[0].values.tolist()
  ndata = ndata.drop(ndata.index[0], axis=0)
  ndata.reset_index(inplace=True)

  return ndata

In [7]:
def data_preprocessing(data1=None, data2=None):
  both = pd.concat([data1, data2], axis = 0)
  area_name = both[both.columns[1]].values[0]
  df = both.transpose()
  df.columns = ['남', '여']
  df.reset_index(inplace=True)
  df = df.drop(df.index[[0, 1, 2, 3, -1]], axis=0)
  df.reset_index(drop=True, inplace=True)
  for i in range(1, 3):
    df[df.columns[i]] = df[df.columns[i]].apply(lambda x: int(x.replace(',', '')))
    df[df.columns[i]] = df[df.columns[i]] / df[df.columns[i]][0]
  
  return df, area_name

In [8]:
def add_graph(data=None):
  fig.add_trace(go.Bar(y= data['index'][1:], x = data['남'][1:], name = 'Male', orientation = 'h',
                       text=['(남성) {} | {}'.format(str(i)[:5], j) for i, j in zip(data['남'][1:], data['index'][1:])], hoverinfo='text'))
  fig.add_trace(go.Bar(y= data['index'][1:], x = data['여'][1:] * -1, name = 'Female', orientation = 'h',
                       text=['(여성) {} | {}'.format(str(i)[:5], j) for i, j in zip(data['여'][1:], data['index'][1:])], hoverinfo='text'))

In [9]:
def tick_lists(data1=None, data2=None):
  df1_point = round(data2['남'][1:].max(), 1)
  df2_point = round(data2['남'][1:].max(), 1)
  if df1_point > df2_point:
    x1 = df1_point
  else:
    x1 = df2_point
  tickv, tickt = [], []
  for i in [-1.25, -2, -4, 0, 4, 2, 1.25]:
    if i == 0:
      tickv.append(0)
      tickt.append('0')
    else:
      tickv.append(x1/i)
      tickt.append(str(abs(x1/i)))
  return tickv, tickt


In [10]:
#성별로 구분 하기
pop_tot = pop[pop.columns[:25]].copy()
pop_male = pop[pop.columns[25:48]].copy()
pop_female = pop[pop.columns[48:]].copy()
insert_columns(pop_male, pop)
insert_columns(pop_female, pop)

In [11]:
pop_tot = data_adjust(pop_tot)
pop_male = data_adjust(pop_male)
pop_female = data_adjust(pop_female)

In [12]:
def find_target_and_others(target_area=None, data1=None, data2=None, data3=None):
  ttot = data1[data1['행정기관'].str.contains(target_area) == True].head(1) #head(1)을 안하면 중복데이터로 실행부분에서 문제가 생길수 있음
  tmale = data2[data2['행정기관'].str.contains(target_area) == True].head(1)
  tfemale = data3[data3['행정기관'].str.contains(target_area) == True].head(1)

  #데이터 정규화 작업
  ttdf = data_normalization(data=ttot, columns= ['전체'], idx=[0, 1, 2, 3])

  #중복회피를 위해 기타지역 데이터프레임 생성
  nttot = pop_tot[pop_tot['행정기관'].str.contains(target_area) == False]

  #데이터 정규화 작업
  nttdf = data_normalization(data=nttot, columns=nttot['행정기관'].values.tolist(), idx=[0, 1, 2, 3])

  #정규화된 데이터 비교 및 제곱오차합계 작성 준비 작업
  nt= prepare_data_comparing(data=nttdf)
  t = prepare_data_comparing(data=ttdf)

  #제곱오차합계 비교작업
  nt['제곱오차합계'] = 0
  for i in t.columns[2:]:
    nt['제곱오차합계'] = nt['제곱오차합계'] + (nt[i] - t[i][0])**2
  min = nt.loc[nt['제곱오차합계'] == nt['제곱오차합계'].min()]
  max = nt.loc[nt['제곱오차합계'] == nt['제곱오차합계'].max()]
  find_min = min['index'].values[0]
  find_max = max['index'].values[0]
  
  #비교데이터셋 준비
  ctmale = pop_male[pop_male['행정기관'].str.contains(find_min) == True].head(1)
  ctfemale = pop_female[pop_female['행정기관'].str.contains(find_min) == True].head(1)
  dtmale = pop_male[pop_male['행정기관'].str.contains(find_max) == True].head(1)
  dtfemale = pop_female[pop_female['행정기관'].str.contains(find_max) == True].head(1)

  return tmale, tfemale, ctmale, ctfemale, dtmale, dtfemale

In [13]:
def data_visualization():
  fig.add_trace(go.Bar(y= df1['index'][1:], x = df1['남'][1:], name = df1_name.split()[-1]+' 남', orientation = 'h', #marker_color='powderblue',
                       text=['(남성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df1['남'][1:], df1['index'][1:])],  hoverinfo='text'),row=1, col=1)
  fig.add_trace(go.Bar(y= df1['index'][1:], x = df1['여'][1:] * -1, name = df1_name.split()[-1]+' 여', orientation = 'h', #marker_color='seagreen',
                       text=['(여성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df1['여'][1:], df1['index'][1:])], hoverinfo='text'),row=1, col=1)
  fig.add_trace(go.Bar(y= df2['index'][1:], x = df2['남'][1:], name = df2_name.split()[-1]+' 남', orientation = 'h', #marker_color='teal',
                       text=['(남성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df2['남'][1:], df2['index'][1:])], hoverinfo='text', opacity=0.5),row=1, col=1)
  fig.add_trace(go.Bar(y= df2['index'][1:], x = df2['여'][1:] * -1, name = df2_name.split()[-1]+' 여', orientation = 'h', #marker_color='darkgreen',
                       text=['(여성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df2['여'][1:], df2['index'][1:])], hoverinfo='text', opacity=0.5),row=1, col=1)
  
  fig.add_trace(go.Bar(y= df1['index'][1:], x = df1['남'][1:], name = df1_name.split()[-1]+' 남', orientation = 'h', #marker_color='powderblue',
                       text=['(남성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df1['남'][1:], df1['index'][1:])],  hoverinfo='text'),row=1, col=2)
  fig.add_trace(go.Bar(y= df1['index'][1:], x = df1['여'][1:] * -1, name = df1_name.split()[-1]+' 여', orientation = 'h', #marker_color='seagreen',
                       text=['(여성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df1['여'][1:], df1['index'][1:])], hoverinfo='text'),row=1, col=2)
  fig.add_trace(go.Bar(y= df3['index'][1:], x = df3['남'][1:], name = df3_name.split()[-1]+' 남', orientation = 'h', #marker_color='teal',
                       text=['(남성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df3['남'][1:], df3['index'][1:])], hoverinfo='text', opacity=0.5),row=1, col=2)
  fig.add_trace(go.Bar(y= df3['index'][1:], x = df3['여'][1:] * -1, name = df3_name.split()[-1]+' 여', orientation = 'h', #marker_color='darkgreen',
                       text=['(여성) {} | {}'.format(str(i)[:5], j) for i, j in zip(df3['여'][1:], df3['index'][1:])], hoverinfo='text', opacity=0.5),row=1, col=2)
  
  fig.update_layout(title_text="{}의 인구구조 비교".format(df1_name.rstrip()), title_font_size = 20, barmode = 'overlay', bargap = 0.10,
                    xaxis = dict(tickvals = tickv1, ticktext = tickt1), xaxis2 = dict(tickvals = tickv2, ticktext = tickt2))

#연도별 지역 인구구조 비교 그래프(실행후 자동 재실행)

In [18]:
#@title 찾고자하는 지역을 입력해주세요 { run: "auto" }
target_area = '\uC6D4\uB871\uBA74' #@param {type:"string"}
print('{}년, {} 을(를) 입력했습니다.'.format(target_year, target_area))
if target_year < 2013 and '세종' in target_area:
  raise ValueError('2013년 이전 데이터는 세종특별자치시가 없습니다.')

tmale, tfemale, ctmale, ctfemale, dtmale, dtfemale = find_target_and_others(target_area=target_area, data1=pop_tot, data2=pop_male, data3=pop_female)
df1, df1_name = data_preprocessing(data1=tmale, data2=tfemale)
df2, df2_name = data_preprocessing(data1=ctmale, data2=ctfemale)
df3, df3_name  = data_preprocessing(data1=dtmale, data2=dtfemale)
tickv1, tickt1 = tick_lists(data1=df1, data2=df2)
tickv2, tickt2 = tick_lists(data1=df1, data2=df3)
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('{} ≒  {}'.format(df1_name, df2_name), '{} ≠  {}'.format(df1_name, df3_name)))
data_visualization()
fig.show()

2012년, 월롱면 을(를) 입력했습니다.
